<a href="https://colab.research.google.com/github/royanlord/DL_Image-Classification_Pneumonia/blob/main/DL_Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


latihan yang dipelajari meliputi:
    1. preprocessing
    2. load gambar dalam sebuah folder
    3. spliting dataset train test
    4. klasifikasi model evaluasi
    5. Find best parameter

In [14]:
# --- 1.1 Instalasi dan Konfigurasi Kaggle ---
!pip install kaggle opendatasets
!pip install opencv-python tensorflow scikit-learn

# Mengunggah kaggle.json
from google.colab import files
import os

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

# Upload file kaggle.json Anda
print("Silakan unggah file kaggle.json Anda:")
uploaded = files.upload()

# Pindahkan file ke direktori konfigurasi Kaggle
for fn in uploaded.keys():
    !mv "$fn" "/root/.kaggle/$fn"
    !chmod 600 /root/.kaggle/$fn

print("Konfigurasi Kaggle selesai.")

Silakan unggah file kaggle.json Anda:


Saving kaggle.json to kaggle.json
Konfigurasi Kaggle selesai.


In [19]:
# --- 1.2 Impor Pustaka ---
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import Xception # Feature Extractor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight
import numpy as np
import cv2
import os
import math
import shutil

# --- 1.3 Unduh Dataset Baseline (Pediatrik Guangzhou) ---
# Gunakan identifier dataset Kaggle
kaggle_url = 'paultimothymooney/chest-xray-pneumonia'
print(f"Mengunduh dataset dari Kaggle: {kaggle_url}")

# Menggunakan opendatasets (membutuhkan otentikasi Kaggle di atas)
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Ekstrak file zip
!unzip -q chest-xray-pneumonia.zip -d chest_xray_data

# Konfigurasi Global dan Path
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 2

# Path Dataset (pastikan sesuai dengan struktur setelah unzip)
BASE_DIR = 'chest_xray_data/chest_xray'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

# Pemeriksaan Data
print(f"\nDataset Train Normal: {len(os.listdir(os.path.join(TRAIN_DIR, 'NORMAL')))} citra")
print(f"Dataset Train Pneumonia: {len(os.listdir(os.path.join(TRAIN_DIR, 'PNEUMONIA')))} citra")

Mengunduh dataset dari Kaggle: paultimothymooney/chest-xray-pneumonia
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace chest_xray_data/chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
Dataset Train Normal: 1341 citra
Dataset Train Pneumonia: 3875 citra


In [20]:
# --- 2.1 Fungsi Peningkatan Citra (CLAHE) ---
def apply_clahe(img):
    """Menerapkan CLAHE dan mengkonversikannya kembali ke 3-channel (RGB)."""

    if isinstance(img, tf.Tensor):
        img_array = img.numpy().astype(np.uint8)
    else:
        # Konversi ke uint8 jika diperlukan (asumsi input dari generator adalah 0-255 atau float)
        img_array = (np.array(img) * 255).astype(np.uint8) if img.dtype != np.uint8 else np.array(img)

    # Konversi ke Grayscale
    if img_array.ndim == 3 and img_array.shape[-1] == 3:
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    else:
        gray = img_array

    # Terapkan CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(gray)

    # Konversi kembali ke 3-channel (RGB)
    clahe_rgb = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2RGB)

    return clahe_rgb

In [21]:
# --- 2.2 Custom Preprocessing dan Data Generator ---
def custom_preprocessing(img):
    """Pipeline Preprocessing: CLAHE -> Normalisasi -> Preprocessing Xception."""

    img_clahe = apply_clahe(img)

    # Normalisasi (0-1)
    img_normalized = img_clahe / 255.0

    # Preprocessing Xception (didefinisikan di sini untuk Keras API)
    return tf.keras.applications.xception.preprocess_input(img_normalized * 255.0)

# Definisi Generator
train_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing
)

# Iterator Data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5216 images belonging to 2 classes.
Found 784 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [22]:
# --- RUN THIS CELL UNTUK DIAGNOSA ---

# 1. Cek jumlah sampel
num_val_samples = val_generator.samples
print(f"Total Sampel di Validation Set: {num_val_samples}")

# 2. Cek komposisi kelas (Harus menunjukkan 2 kelas: Normal & Pneumonia)
class_indices = val_generator.class_indices
class_names = list(class_indices.keys())
print(f"Nama Kelas di Generator: {class_names}")

# 3. Hitung distribusi kelas (Penting!)
from collections import Counter
val_labels = val_generator.classes
class_counts = Counter(val_labels)
print("\nDistribusi Kelas di Validation Set:")
for index, count in class_counts.items():
    print(f"  {class_names[index]}: {count} sampel")

# --- RUN THIS CELL UNTUK DIAGNOSA ---

Total Sampel di Validation Set: 784
Nama Kelas di Generator: ['NORMAL', 'PNEUMONIA']

Distribusi Kelas di Validation Set:
  NORMAL: 202 sampel
  PNEUMONIA: 582 sampel


In [23]:
# --- 2.3 Perhitungan Class Weights ---
class_labels = train_generator.classes
class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weights_dict = dict(enumerate(class_weights_array))

print(f"\nBobot Kelas (0=NORMAL, 1=PNEUMONIA): {class_weights_dict}")


Bobot Kelas (0=NORMAL, 1=PNEUMONIA): {0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}


In [29]:
# --- 3.1 Definisi Model (Enhanced PneuX-Net) ---
from tensorflow.keras.regularizers import l2

def build_enhanced_pneumo_net(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=2):

    # 1. Feature Extractor (Xception)
    base_xception = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze the base layers for the first phase
    for layer in base_xception.layers:
        layer.trainable = False

    inputs = Input(shape=input_shape)
    x = base_xception(inputs, training=False)
    x = GlobalAveragePooling2D()(x)

    # 2. Classification Head (EfficientNet-inspired replacement for KNC)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    # x = Dropout(0.5)(x)
    x = Dropout(0.6)(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    # x = Dropout(0.3)(x)
    x = Dropout(0.5)(x)

    # 3. Output Layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs, name="Enhanced_PneuX_Net")

    return model

model = build_enhanced_pneumo_net()
model.summary()

Model: "Enhanced_PneuX_Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,976,490 (83.83 MB)

 Trainable params: 1,115,010 (4.25 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [30]:
# --- 3.2 Pelatihan (Fase I) ---
LEARNING_RATE_PHASE1 = 1e-4
EPOCHS_PHASE1 = 20

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_PHASE1),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'), # Metrik Klinis
        tf.keras.metrics.AUC(name='auc')        # Metrik Klinis
    ]
)

# Callbacks
checkpoint_filepath = 'best_model_phase1.weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_recall', # Fokus pada sensitivitas
    mode='max',
    save_best_only=True
)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\n--- Memulai Pelatihan Fase 1: Feature Extraction Training ---")
history = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)

# Load bobot terbaik
model.load_weights(checkpoint_filepath)


--- Memulai Pelatihan Fase 1: Feature Extraction Training ---
Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 134s 761ms/step - accuracy: 0.6883 - auc: 0.7538 - loss: 1.5927 - precision: 0.6883 - recall: 0.6883 - val_accuracy: 0.7423 - val_auc: 0.7176 - val_loss: 1.9032 - val_precision: 0.7423 - val_recall: 0.7423
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 715ms/step - accuracy: 0.8107 - auc: 0.8784 - loss: 1.3436 - precision: 0.8107 - recall: 0.8107 - val_accuracy: 0.7423 - val_auc: 0.7342 - val_loss: 1.8730 - val_precision: 0.7423 - val_recall: 0.7423
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 115s 705ms/step - accuracy: 0.8211 - auc: 0.8872 - loss: 1.2466 - precision: 0.8211 - recall: 0.8211 - val_accuracy: 0.7423 - val_auc: 0.7317 - val_loss: 1.6785 - val_precision: 0.7423 - val_recall: 0.7423
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 712ms/step - accuracy: 0.8326 - auc: 0.9016 - loss: 1.1449 - precision: 0.8326 - recall: 0.8326 - val_accuracy: 0.7423 - val_auc: 0.7360 - val_loss: 1.749

In [31]:
# --- 3.3 Fine-Tuning (Fase II) ---
from tensorflow.keras.callbacks import ReduceLROnPlateau

LEARNING_RATE_PHASE2 = 1e-5 # Sangat kecil

# Unfreeze lapisan Xception teratas (misalnya, 20 lapisan terakhir)
for layer in model.layers[1].layers[-20:]:
    layer.trainable = True

# Kompilasi ulang dengan Learning Rate yang lebih rendah
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_PHASE2),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall', 'auc']
)

# --- 2. Callbacks Baru ---
checkpoint_filepath_ft = 'best_model_fine_tuning.weights.h5'
model_checkpoint_callback_ft = ModelCheckpoint(
    filepath=checkpoint_filepath_ft,
    save_weights_only=True,
    monitor='val_auc', # Monitor AUC karena lebih stabil
    mode='max',
    save_best_only=True
)

early_stopping_callback_ft = EarlyStopping(
    monitor='val_auc',
    patience=7, # Beri sedikit lebih banyak waktu
    mode='max',
    restore_best_weights=True
)

# INOVASI: Learning Rate Scheduler
reduce_lr = ReduceLROnPlateau(
    monitor='val_auc',
    factor=0.2,           # Kurangi LR menjadi 20% dari nilai saat ini
    patience=3,           # Tunggu 3 epoch tanpa peningkatan val_auc
    min_lr=1e-8,          # Batas LR minimum yang sangat rendah
    mode='max',
    verbose=1
)

# --- 3. Pelatihan ---
print("\n--- Memulai Pelatihan Fase 2: Fine-Tuning dengan Regularisasi Kuat ---")
history_ft = model.fit(
    train_generator,
    epochs=50, # Tingkatkan jumlah epoch total (karena LR sangat rendah)
    initial_epoch=history.epoch[-1],
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[model_checkpoint_callback_ft, early_stopping_callback_ft, reduce_lr]
)

# Load bobot terbaik dari fine-tuning
model.load_weights(checkpoint_filepath_ft)


--- Memulai Pelatihan Fase 2: Fine-Tuning dengan Regularisasi Kuat ---
Epoch 13/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 146s 789ms/step - accuracy: 0.7933 - auc: 0.8685 - loss: 1.4643 - precision: 0.7933 - recall: 0.7933 - val_accuracy: 0.7423 - val_auc: 0.7723 - val_loss: 1.7231 - val_precision: 0.7423 - val_recall: 0.7423 - learning_rate: 1.0000e-05
Epoch 14/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 736ms/step - accuracy: 0.8232 - auc: 0.8896 - loss: 1.3215 - precision: 0.8232 - recall: 0.8232 - val_accuracy: 0.7423 - val_auc: 0.7855 - val_loss: 1.5962 - val_precision: 0.7423 - val_recall: 0.7423 - learning_rate: 1.0000e-05
Epoch 15/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 140s 726ms/step - accuracy: 0.8325 - auc: 0.9051 - loss: 1.2803 - precision: 0.8325 - recall: 0.8325 - val_accuracy: 0.7423 - val_auc: 0.7844 - val_loss: 1.6003 - val_precision: 0.7423 - val_recall: 0.7423 - learning_rate: 1.0000e-05
Epoch 16/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 723ms/step - accuracy: 0.8338 - auc: 0.9038 - loss: 1.269

In [32]:
# --- SIMPAN MODEL FINAL KE FORMAT .keras ---

# Gunakan format native Keras (.keras) yang direkomendasikan
FINAL_MODEL_FILENAME = 'enhanced_pneumo_net_final.keras'

# Simpan seluruh model (arsitektur, bobot, optimizer, dll.)
model.save(FINAL_MODEL_FILENAME)

print(f"Model final telah disimpan sebagai: {FINAL_MODEL_FILENAME}")

Model final telah disimpan sebagai: enhanced_pneumo_net_final.keras


In [33]:
# --- 4.1 Evaluasi Internal ---
print("\n--- Evaluasi Akhir pada Dataset Test Internal (Guangzhou) ---")
results_internal = model.evaluate(test_generator)

print(f"\nLoss Test: {results_internal[0]:.4f}")
print(f"Accuracy Test: {results_internal[1]:.4f}")
print(f"Recall Test: {results_internal[3]:.4f}")
print(f"AUC Test: {results_internal[4]:.4f}")

# Simpan hasil untuk visualisasi plot
# Plot akurasi dan loss history


--- Evaluasi Akhir pada Dataset Test Internal (Guangzhou) ---
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 259ms/step - accuracy: 0.3012 - auc: 0.3826 - loss: 3.2651 - precision: 0.3012 - recall: 0.3012

Loss Test: 2.1498
Accuracy Test: 0.6266
Recall Test: 0.6266
AUC Test: 0.7510
